<a href="https://colab.research.google.com/github/Van-Wu1/cycle/blob/main/scr/py/final_CQI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [28]:
import math
import numpy as np
from shapely.geometry import LineString, MultiLineString
from tqdm import tqdm
import geopandas as gpd
import pandas as pd
from shapely.geometry import Point
from shapely.strtree import STRtree
import matplotlib.pyplot as plt
import os
import glob
from shapely.ops import linemerge
import hashlib

In [29]:
from google.colab import drive
drive.mount('/content/drive')
!ls '/content/drive/MyDrive/CASA0004_Cycling/data'

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
BoroughShp  CQI  GreatLondonShp  s1  s2_Env  s3


In [30]:
s1 = gpd.read_file("/content/drive/MyDrive/CASA0004_Cycling/data/CQI/index_s1.gpkg")
print(s1.columns)

s2 = gpd.read_file("/content/drive/MyDrive/CASA0004_Cycling/data/CQI/index_s2.gpkg")
print(s2.columns)

s3 = gpd.read_file("/content/drive/MyDrive/CASA0004_Cycling/data/CQI/index_s3.gpkg")
print(s3.columns)

Index(['id', 'name', 'way_type', 'index', 'index_10', 'stress_level', 'offset',
       'side', 'proc_width', 'proc_surface', 'proc_smoothness', 'proc_oneway',
       'proc_sidepath', 'proc_highway', 'proc_maxspeed',
       'proc_traffic_mode_left', 'proc_traffic_mode_right',
       'proc_separation_left', 'proc_separation_right', 'proc_buffer_left',
       'proc_buffer_right', 'proc_mandatory', 'proc_traffic_sign', 'fac_width',
       'fac_surface', 'fac_highway', 'fac_maxspeed', 'base_index', 'fac_1',
       'fac_2', 'fac_3', 'fac_4', 'data_bonus', 'data_malus',
       'data_incompleteness', 'data_missing', 'data_missing_width',
       'data_missing_surface', 'data_missing_smoothness',
       'data_missing_maxspeed', 'data_missing_parking', 'data_missing_lit',
       'filter_usable', 'filter_way_type', 'proc_slope', 'fac_5', 'layer',
       'path', 'geometry'],
      dtype='object')
Index(['id', 'name', 'way_type', 'gvi_final', 'fac_gvi', 'NO2_mean', 'fac_no2',
       'natural_score',

### 又他妈的重复id了，很好，排查了一下不是bug，是osm的共用geometry

In [31]:
'''s1["id"].value_counts().head(3)
s2["id"].value_counts().head(3)
s3["id"].value_counts().head(3)'''

's1["id"].value_counts().head(3)\ns2["id"].value_counts().head(3)\ns3["id"].value_counts().head(3)'

In [32]:
cnt1 = s1.groupby("id").size()
cnt2 = s2.groupby("id").size()
cnt3 = s3.groupby("id").size()
check = pd.concat([cnt1.rename("n1"), cnt2.rename("n2"), cnt3.rename("n3")], axis=1).fillna(0).astype(int)
check.sample(5)

,n1,n2,n3
id,,,
way/652249275,1,1,1
way/26762784,1,1,1
way/3873020,1,1,1
way/311644343,1,1,1
way/1376966570,1,1,1


In [33]:
# 1) 统一 CRS（建议米制；若已有米制，可注释掉）
TARGET_CRS = "EPSG:27700"
for _g in [s1, s2, s3]:
    if _g.crs != TARGET_CRS:
        _g.to_crs(TARGET_CRS, inplace=True)

# 2) 为每条线段构造稳定的“段级唯一键 edge_uid”
def make_edge_uid(gdf, precision=3):
    def uid_from_geom(geom):
        if geom is None or geom.is_empty:
            return None

        # 统一处理成“两个端点”
        def endpoints_of_linestring(ls: LineString):
            cs = list(ls.coords)
            return (cs[0], cs[-1])

        start, end = None, None

        if isinstance(geom, LineString):
            start, end = endpoints_of_linestring(geom)

        elif isinstance(geom, MultiLineString):
            # 优先尝试将多段合并成一条
            merged = linemerge(geom)
            if isinstance(merged, LineString):
                start, end = endpoints_of_linestring(merged)
            else:
                # 仍是 MultiLineString：收集所有端点，取“全局最小/最大”端点作为代表
                eps = []
                for part in geom.geoms:
                    if isinstance(part, LineString) and not part.is_empty:
                        a, b = endpoints_of_linestring(part)
                        eps.extend([a, b])
                if not eps:
                    return None
                # 四舍五入后进行无向排序
                eps = [(round(x, precision), round(y, precision)) for (x, y) in eps]
                a = min(eps)  # 词典序最小
                b = max(eps)  # 词典序最大
                start, end = a, b

        else:
            return None

        # 无向排序 + 四舍五入
        x1, y1 = start
        x2, y2 = end
        a = (round(min(x1, x2), precision), round(min(y1, y2), precision))
        b = (round(max(x1, x2), precision), round(max(y1, y2), precision))
        raw = f"{a[0]},{a[1]}|{b[0]},{b[1]}"
        return hashlib.sha1(raw.encode()).hexdigest()[:16]

    gdf = gdf.copy()
    gdf["edge_uid"] = gdf.geometry.map(uid_from_geom)
    return gdf


s1 = make_edge_uid(s1)
s2 = make_edge_uid(s2)
s3 = make_edge_uid(s3)

# 3) 同一数据内如仍有相同 (id, edge_uid) 的重复记录，保留“长度最长”的那条（最保守）
def keep_longest_per_uid(gdf, length_col=None):
    """
    对同一 (id, edge_uid) 的重复记录，只保留长度最长的一条。
    - 如果 length_col 为空或不存在，则用 geometry.length 作为长度。
    - 安全地处理缺失 edge_uid / 空几何。
    """
    df = gdf.copy()

    # 1) 确保有长度列
    temp_len_col = None
    if (length_col is None) or (length_col not in df.columns):
        temp_len_col = "_len_tmp_"
        # geometry 可能有空，length 会得到 NaN；无妨
        df[temp_len_col] = df.geometry.length
        length_col = temp_len_col

    # 2) 防止缺键
    if "id" not in df.columns or "edge_uid" not in df.columns:
        raise ValueError("需要列 'id' 和 'edge_uid' 才能去重。")

    # 3) 排序 + 分组取每组第一条（长度最大）
    df["_row"] = np.arange(len(df))
    df = df.sort_values(["id", "edge_uid", length_col], ascending=[True, True, False])

    idx = (
        df.groupby(["id", "edge_uid"], dropna=False)["_row"]
          .head(1)
          .values
    )

    out = df.loc[idx].drop(columns=["_row"], errors="ignore")

    # 4) 清理临时列
    if temp_len_col is not None:
        out = out.drop(columns=[temp_len_col], errors="ignore")

    return out

s1 = keep_longest_per_uid(s1, length_col="length_m" if "length_m" in s1.columns else None)
s2 = keep_longest_per_uid(s2, length_col="length_m" if "length_m" in s2.columns else None)
s3 = keep_longest_per_uid(s3, length_col="length_m" if "length_m" in s3.columns else None)

# 4) 选择用于连接的键：优先 (id, edge_uid)，若 edge_uid 为空则退化为 id
#    为了实现“优先 uid，缺失时 id”，这里分别做两次合并再合成
key_cols = ["id","edge_uid"]

# —— 先把需要的几何备份：以 s1 的几何为主，s2/s3 的几何以 WKT 形式保留（可选）
s2_geom_wkt = s2.geometry.to_wkt()
s3_geom_wkt = s3.geometry.to_wkt()
s2 = s2.drop(columns="geometry").assign(geometry_s2=s2_geom_wkt.values)
s3 = s3.drop(columns="geometry").assign(geometry_s3=s3_geom_wkt.values)

# 5) 外连接：s1 ⟂ s2（先用 (id, edge_uid)）
m12 = s1.merge(s2, on=key_cols, how="outer", suffixes=("_s1","_s2"))

# 对仍未匹配到 edge_uid 的记录，尝试用 id 再补一次（避免同 id 的孤段丢失）
# 标记哪些在第一次 merge 里没匹配成功
def mark_unmatched(df, left_prefix="_left", right_prefix="_right"):
    # 粗略：存在任一 s2 字段则视为匹配
    cols_s2 = [c for c in df.columns if c.endswith("_s2") or c=="geometry_s2"]
    matched = df[cols_s2].notna().any(axis=1)
    return ~matched

mask_unmatched_s2 = mark_unmatched(m12)
if mask_unmatched_s2.any():
    # 用 id 合并补齐剩余
    add2 = s2.merge(s1[["id"]].drop_duplicates(), on="id", how="right")
    m12_fallback = s1.merge(add2, on="id", how="left", suffixes=("_s1","_s2_fb"))
    # 仅对原来没匹配到的行，用 fallback 的 s2 字段填充
    cols_fb = [c for c in m12_fallback.columns if c.endswith("_s2_fb") or c=="geometry_s2"]
    for c in cols_fb:
        base = c.replace("_s2_fb","_s2")
        if base in m12.columns:
            m12.loc[mask_unmatched_s2, base] = m12_fallback.loc[mask_unmatched_s2, c]
        else:
            m12[base] = m12_fallback.loc[mask_unmatched_s2, c]
    # 清理临时列
    drop_fb = [c for c in m12.columns if c.endswith("_s2_fb")]
    m12.drop(columns=drop_fb, inplace=True, errors="ignore")

# 6) 再把 s3 并进来（同样策略）
m123 = m12.merge(s3, on=key_cols, how="outer")  # s3 已带 _s3 后缀的字段
# 对没匹配到 s3 的，再用 id 兜底
cols_s3 = [c for c in m123.columns if c.endswith("_s3") or c=="geometry_s3"]
mask_unmatched_s3 = ~m123[cols_s3].notna().any(axis=1)
if mask_unmatched_s3.any():
    add3 = s3.merge(m12[["id"]].drop_duplicates(), on="id", how="right")
    m123_fb = m12.merge(add3, on="id", how="left")
    # 用 fallback 填
    for c in [c for c in m123_fb.columns if c.endswith("_s3") or c=="geometry_s3"]:
        if c in m123.columns:
            m123.loc[mask_unmatched_s3, c] = m123_fb.loc[mask_unmatched_s3, c]
        else:
            m123[c] = m123_fb.loc[mask_unmatched_s3, c]

# 7) 结果整理
# - 保留 s1 的 geometry 作为主几何
out = gpd.GeoDataFrame(m123, geometry="geometry", crs=s1.crs)

# 8) 导出
OUT_GPKG = "/content/drive/MyDrive/CASA0004_Cycling/data/CQI/merged_s1_s2_s3.gpkg"
out.to_file(OUT_GPKG, layer="merged_all_fields", driver="GPKG")
print(f"已导出：{OUT_GPKG}  | 行数：{len(out)}")


已导出：/content/drive/MyDrive/CASA0004_Cycling/data/CQI/merged_s1_s2_s3.gpkg  | 行数：163104


In [34]:
out["geometry"].value_counts().head(10)

,count
geometry,
"LINESTRING (530854 179502, 530862 179506, 5308...",1
"LINESTRING (530622 196188, 530614 196230, 5306...",1
"LINESTRING (530710 171664, 530724 171658, 5307...",1
"LINESTRING (530704 171666, 530710 171664)",1
"LINESTRING (515654 180186, 515656 180186, 5156...",1
"LINESTRING (543330 165134, 543318 165144, 5433...",1
"LINESTRING (530944 179180, 530916 179198)",1
"LINESTRING (530996 179318, 530942 179348)",1
"LINESTRING (530942 179348, 530938 179350, 5309...",1


In [35]:
print(out.columns)

Index(['id', 'name_s1', 'way_type_s1', 'index', 'index_10', 'stress_level',
       'offset', 'side', 'proc_width', 'proc_surface', 'proc_smoothness',
       'proc_oneway', 'proc_sidepath', 'proc_highway', 'proc_maxspeed',
       'proc_traffic_mode_left', 'proc_traffic_mode_right',
       'proc_separation_left', 'proc_separation_right', 'proc_buffer_left',
       'proc_buffer_right', 'proc_mandatory', 'proc_traffic_sign', 'fac_width',
       'fac_surface', 'fac_highway', 'fac_maxspeed', 'base_index', 'fac_1',
       'fac_2', 'fac_3', 'fac_4', 'data_bonus', 'data_malus',
       'data_incompleteness', 'data_missing', 'data_missing_width',
       'data_missing_surface', 'data_missing_smoothness',
       'data_missing_maxspeed', 'data_missing_parking', 'data_missing_lit',
       'filter_usable', 'filter_way_type', 'proc_slope', 'fac_5', 'layer',
       'path', 'geometry', 'edge_uid', 'name_s2', 'way_type_s2', 'gvi_final',
       'fac_gvi', 'NO2_mean', 'fac_no2', 'natural_score', 'fac_nat', 

In [36]:
keep_cols = [
    "id", "name_s1", "way_type_s1", "length_m",
    "index", "base_index", "fac_1", "fac_2", "fac_3", "fac_4", "fac_5", "stress_level",
    "index_2", "fac_gvi", "fac_no2", "fac_nat",
    "index_3", "edge_betweenness", "edge_closeness_2km", "edge_closeness_5km", "_B_log", "B_norm", "C2_norm", "C5_norm", "C_multi",
    "geometry"
]

# 按列顺序保留
out_gdf = out[keep_cols]

# 重命名
out_gdf = out_gdf.rename(columns={
    "name_s1": "name",
    "way_type_s1": "way_type",
    "index": "index_1",
    "stress_level": "LTS"
})

# seesee
print(out_gdf.columns)

Index(['id', 'name', 'way_type', 'length_m', 'index_1', 'base_index', 'fac_1',
       'fac_2', 'fac_3', 'fac_4', 'fac_5', 'LTS', 'index_2', 'fac_gvi',
       'fac_no2', 'fac_nat', 'index_3', 'edge_betweenness',
       'edge_closeness_2km', 'edge_closeness_5km', '_B_log', 'B_norm',
       'C2_norm', 'C5_norm', 'C_multi', 'geometry'],
      dtype='object')


In [37]:
# 删除 index_1 / index_2 / index_3 任意一个为空的行
out_gdf = out_gdf.dropna(subset=['index_1', 'index_2', 'index_3'], how='any')

# 重置索引
out_gdf = out_gdf.reset_index(drop=True)

## 很好弄干净了，搞一点敏感性分析

The Composite Cycling Quality Index (CQI) was constructed by aggregating three sub-indices (Index_1, Index_2, and Index_3) representing infrastructure quality, environmental perception, and network centrality, respectively. The baseline CQI was calculated as the simple arithmetic mean of the three sub-indices to ensure equal contribution from each dimension. To assess the robustness of the CQI, a sensitivity analysis was performed by varying the weights assigned to each sub-index (e.g., Index_1:Index_2:Index_3 = 0.5:0.3:0.2, 0.4:0.4:0.2, 0.5:0.4:0.1). The sensitivity analysis allows examination of how changes in the relative importance of each dimension influence the final CQI distribution and spatial patterns, thereby evaluating the stability and reproducibility of the composite measure.

综合骑行质量指数（CQI）是通过将三个子指数（分别代表基础设施质量、环境感知和网络中心性）进行聚合而构建的。基准 CQI 是这三个子指数的简单算术平均值，以确保每个维度的贡献相同。为了评估 CQI 的稳健性，进行了敏感性分析，通过改变每个子指数所赋予的权重（例如，Index_1：Index_2：Index_3 = 0.5：0.3：0.2，0.4：0.4：0.2，0.5：0.4：0.1）来进行。敏感性分析允许考察每个维度相对重要性的变化如何影响最终的 CQI 分布和空间模式，从而评估综合衡量指标的稳定性和可重复性。

### 还得改啊，再做一下s3会不会很影响

In [46]:
# === 1. 构建总的CQI指标（基础版：三个子指数的等权平均） ===
out_gdf['cqi_index'] = out_gdf[['index_1', 'index_2', 'index_3']].mean(axis=1)

# 检查基本统计
print("CQI Index 基本统计：")
print(out_gdf['cqi_index'].describe())

# === 2. 为敏感性分析留出空间（例如调整权重） ===
# 定义一个函数，可以快速测试不同权重组合的结果
def calc_cqi(df, w1=1/3, w2=1/3, w3=1/3):
    """根据权重计算CQI"""
    return df['index_1']*w1 + df['index_2']*w2 + df['index_3']*w3

# 示例：偏重 index_1
out_gdf['cqi_index_w1'] = calc_cqi(out_gdf, w1=0.5, w2=0.25, w3=0.25)

# 示例：偏重 index_2
out_gdf['cqi_index_w2'] = calc_cqi(out_gdf, w1=0.25, w2=0.5, w3=0.25)

# 示例：偏重 index_3
out_gdf['cqi_index_w3'] = calc_cqi(out_gdf, w1=0.25, w2=0.25, w3=0.5)

# 输出前几行看结果
print(out_gdf[['id', 'cqi_index', 'cqi_index_w1', 'cqi_index_w2', 'cqi_index_w3']].head())


CQI Index 基本统计：
count    162927.000000
mean          8.769635
std           4.717752
min           0.770626
25%           5.508547
50%           8.167980
75%          10.437605
max          33.599206
Name: cqi_index, dtype: float64
               id  cqi_index  cqi_index_w1  cqi_index_w2  cqi_index_w3
0  way/1000024348   7.126829     10.595122      5.419365      5.366000
1  way/1000024349   6.448047      9.586035      4.901585      4.856520
2  way/1000057960   3.239950      4.679962      2.514194      2.525693
3   way/100009771   5.175731      7.631799      4.011228      3.884168
4   way/100015712  10.757340     16.068005      8.130732      8.073283


In [47]:
print(out_gdf.columns)

Index(['id', 'name', 'way_type', 'length_m', 'index_1', 'base_index', 'fac_1',
       'fac_2', 'fac_3', 'fac_4', 'fac_5', 'LTS', 'index_2', 'fac_gvi',
       'fac_no2', 'fac_nat', 'index_3', 'edge_betweenness',
       'edge_closeness_2km', 'edge_closeness_5km', '_B_log', 'B_norm',
       'C2_norm', 'C5_norm', 'C_multi', 'geometry', 'cqi_index',
       'cqi_index_w1', 'cqi_index_w2', 'cqi_index_w3'],
      dtype='object')


In [48]:
OUT_GPKG = "/content/drive/MyDrive/CASA0004_Cycling/data/CQI/CQI.gpkg"
out.to_file(OUT_GPKG, layer="merged_all_fields", driver="GPKG")
print(f"已导出：{OUT_GPKG}  | 行数：{len(out)}")

已导出：/content/drive/MyDrive/CASA0004_Cycling/data/CQI/CQI.gpkg  | 行数：163104
